In this notebook, we collect the data performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled List of Falcon 9 and Falcon Heavy launches.

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches

## Importing Necessary Libraries

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import unicodedata

## Getting the data

In [2]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

In [3]:
response=requests.get(static_url)
response

<Response [200]>

In [4]:
soup=BeautifulSoup(response.content)
soup.title

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>

In [ ]:
soup

## Extracting the list of columns

In [5]:
html_tables=soup.find_all(name='table')

In [ ]:
# The 3rd table contains the data that we want
# Let's print the third table and check its content
first_launch_table = html_tables[2]
print(first_launch_table)

In [7]:
col_list=first_launch_table.find_all(name='th',scope="col")
col_list

[<th scope="col">Flight No.
 </th>,
 <th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
 </th>,
 <th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
 </th>,
 <th scope="col">Launch site
 </th>,
 <th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
 </th>,
 <th scope="col">Payload mass
 </th>,
 <th scope="col">Orbit
 </th>,
 <th scope="col">Customer
 </th>,
 <th scope="col">Launch<br/>outcome
 </th>,
 <th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
 </th>]

In [8]:
len(first_launch_table.find_all(name='th',scope="col"))

10

In [9]:
columns=[]

In [10]:
for i in range(0,10):
  columns.append(col_list[i].text.strip())
columns=columns[-10:]
columns

['Flight No.',
 'Date andtime (UTC)',
 'Version,Booster [b]',
 'Launch site',
 'Payload[c]',
 'Payload mass',
 'Orbit',
 'Customer',
 'Launchoutcome',
 'Boosterlanding']

## Creating a Dataframe

In [11]:
launch_dict= dict.fromkeys(columns)

# Remove some columns
del launch_dict['Date andtime (UTC)']
del launch_dict['Launchoutcome']
del launch_dict['Boosterlanding']
del launch_dict['Version,Booster [b]']
del launch_dict['Payload[c]']


# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

launch_dict

{'Flight No.': [],
 'Launch site': [],
 'Payload mass': [],
 'Orbit': [],
 'Customer': [],
 'Payload': [],
 'Launch outcome': [],
 'Version Booster': [],
 'Booster landing': [],
 'Date': [],
 'Time': []}

In [12]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


In [13]:
extracted_row = 0
#Extract each table
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
   # get table row
    for rows in table.find_all("tr"):
        #check to see if first table heading is as number corresponding to launch a number
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        #get table element
        row=rows.find_all('td')
        #if it is number save cells in a dictonary
        if flag:
            extracted_row += 1
            # Flight Number value
            # TODO: Append the flight_number into launch_dict with key `Flight No.`
            launch_dict['Flight No.'].append(flight_number)
            #print(flight_number)
            datatimelist=date_time(row[0])

            # Date value
            # TODO: Append the date into launch_dict with key `Date`
            date = datatimelist[0].strip(',')
            launch_dict['Date'].append(date)
            #print(date)

            # Time value
            # TODO: Append the time into launch_dict with key `Time`
            time = datatimelist[1]
            launch_dict['Time'].append(time)
            #print(time)

            # Booster version
            # TODO: Append the bv into launch_dict with key `Version Booster`
            bv=booster_version(row[1])
            if not(bv):
                bv=row[1].a.string
            #print(bv)
            launch_dict['Version Booster'].append(bv)

            # Launch Site
            # TODO: Append the bv into launch_dict with key `Launch Site`
            launch_site = row[2].a.string
            launch_dict['Launch site'].append(launch_site)
            #print(launch_site)

            # Payload
            # TODO: Append the payload into launch_dict with key `Payload`
            payload = row[3].a.string
            launch_dict['Payload'].append(payload)
            #print(payload)

            # Payload Mass
            # TODO: Append the payload_mass into launch_dict with key `Payload mass`
            payload_mass = get_mass(row[4])
            launch_dict['Payload mass'].append(payload_mass)
            #print(payload)

            # Orbit
            # TODO: Append the orbit into launch_dict with key `Orbit`
            orbit = row[5].a.string
            launch_dict['Orbit'].append(orbit)
            #print(orbit)

            # Customer
            # TODO: Append the customer into launch_dict with key `Customer`
            # Assuming that row[6].a is the element you want to extract the string from
            customer_element = row[6].a

# Check if customer_element is not None before accessing its string attribute
            if customer_element is not None:
              customer = customer_element.string
              launch_dict['Customer'].append(customer)
            else:
    # Handle the case when customer_element is None, e.g., append a default value or raise an exception
              launch_dict['Customer'].append("Unknown Customer")
    # Alternatively, you can raise an exception to explicitly handle the unexpected situation
    # raise ValueError("Customer element is None")
            #print(customer)

            # Launch outcome
            # TODO: Append the launch_outcome into launch_dict with key `Launch outcome`
            launch_outcome = list(row[7].strings)[0]
            #print(launch_outcome)
            launch_dict['Launch outcome'].append(launch_outcome)
            # Booster landing
            # TODO: Append the launch_outcome into launch_dict with key `Booster landing`
            booster_landing = landing_status(row[8])
            launch_dict['Booster landing'].append(booster_landing)
            #print(booster_landing)


In [14]:
lengths = [len(value_list) for value_list in launch_dict.values()]
print(lengths)

[121, 121, 121, 121, 121, 121, 121, 121, 121, 121, 121]


In [15]:
df=pd.DataFrame(launch_dict)
df.head()

,Flight No.,Launch site,Payload mass,Orbit,Customer,Payload,Launch outcome,Version Booster,Booster landing,Date,Time
0,1,CCAFS,0,LEO,SpaceX,Dragon Spacecraft Qualification Unit,Success\n,F9 v1.0B0003.1,Failure,4 June 2010,18:45
1,2,CCAFS,0,LEO,NASA,Dragon,Success,F9 v1.0B0004.1,Failure,8 December 2010,15:43
2,3,CCAFS,525 kg,LEO,NASA,Dragon,Success,F9 v1.0B0005.1,No attempt\n,22 May 2012,07:44
3,4,CCAFS,"4,700 kg",LEO,NASA,SpaceX CRS-1,Success\n,F9 v1.0B0006.1,No attempt,8 October 2012,00:35
4,5,CCAFS,"4,877 kg",LEO,NASA,SpaceX CRS-2,Success\n,F9 v1.0B0007.1,No attempt\n,1 March 2013,15:10


## Exporting the data to a CSV file

In [16]:
df.to_csv('spacex_web_scraped.csv', index=False)